### Lo primero que tengo que hacer es cargar el Dataframe del Notebook 2:

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

# Creo una carpeta para mi práctica en el directorio raíz
!mkdir -p "/content/drive/My Drive/Práctica NLP Najli YE"

Mounted at /content/drive


Importo el csv del Notebook anterior:

In [2]:
import pandas as pd
proc_df = pd.read_csv('/content/drive/My Drive/Práctica NLP Najli YE/lux_beauty_processed_reviews.csv', sep=',', decimal='.')

Compruebo que está todo OK:

In [3]:
proc_df.head()

,overall,reviewText,sentiment_label,processedReview
0,1.0,Mixed with water. I use the real rusk shampoo ...,0,mixed water use real rusk shampoo every week same
1,1.0,I have used plumpers before and I can tell you...,0,used plumpers tell product nothing love image ...
2,1.0,The brush is splayed out so i can't really use...,0,brush splayed cant really use it seems like ch...
3,1.0,"This soap is awful, it arrived leaking in its ...",0,soap awful arrived leaking package seeped box ...
4,1.0,does not work,0,work


Ahora instalamos utilidades de la librería `sklearn` (sobre todo) para hacer nuestros modelos

In [4]:
from sklearn.model_selection import train_test_split # Modelado
from sklearn.pipeline import Pipeline # Modelado
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # Modelado


Creamos los conjuntos de entrenamiento (75% del total) y test (25%)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    proc_df['processedReview'], # Reviews procesadas
    proc_df['sentiment_label'], # Cogemos la variable binaria
    train_size=0.75,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

In [6]:
X_train.iloc[:10]

21487      good shave soap prefer green one good runner up
8220     unfortunately agree posters used supposedly we...
21374    light oilfree lotion sunscreen comes pump disp...
12469                            makes skin feel dry itchy
11541    nice color little light me sparkles too know u...
23896                   mustdo step using vinylux polishes
29410    like thin pencil use create short strokes fill...
19621    gets hot quick curl hair older one takes forev...
25407    love stuff much smooth skin kp saw someones re...
9802             works sometimes stops working couple zaps
Name: processedReview, dtype: object

In [7]:
y_train.iloc[:10]

21487    1
8220     0
21374    1
12469    0
11541    0
23896    1
29410    1
19621    0
25407    1
9802     0
Name: sentiment_label, dtype: int64



---



### Ahora es cuando necesitamos convertir estas palabras (texto) a valores numéricos con los que entrenar nuestros modelos de Machine Learning o Deep Learning

Para la representación, escojo `TF-IDF Vectorizer` que entiende mejor el contexto de las palabras en el documento y corpus. Además, aunque las reseñas son de la misma temática (productos de belleza), he visto que la variedad de productos (y por tanto de palabras) es alta.

In [8]:
feat_extr = TfidfVectorizer(
    max_df=0.95,
    min_df=3,
    max_features=3000,
    ngram_range=(1, 3))

Como estoy eligiendo los n-gramas hasta un valor de 3-gramas, elijo un `max_features` alto, que el tamaño de vocabulario considerado sea grande.

Igual que hacíamos en modelos numéricos, esta *extracción de características* la aplico con un `fit` solamente en el conjunto de Train:

In [9]:
# # Aplicamos "fit" de los TF-IDF **SOLO** en conjunto de Train (el conocido)
feat_extr.fit(X_train)

TfidfVectorizer(max_df=0.95, max_features=3000, min_df=3, ngram_range=(1, 3))

Chequeo la longitud del vocabulario:

In [10]:
print(len(feat_extr.vocabulary_))

3000


Algunas palabras:

In [11]:
print(list(feat_extr.vocabulary_.items())[:20])

[('good', 960), ('shave', 2270), ('soap', 2392), ('prefer', 1883), ('green', 991), ('one', 1727), ('up', 2738), ('unfortunately', 2729), ('agree', 47), ('used', 2762), ('version', 2798), ('felt', 807), ('working', 2926), ('tingling', 2633), ('etc', 694), ('supposed', 2527), ('stronger', 2499), ('feel', 796), ('nothing', 1683), ('perhaps', 1827)]


In [12]:
# # TF-IDF Lo aplicamos con "transform" al conjunto de train y de test:
X_train_ = feat_extr.transform(X_train)
X_test_ = feat_extr.transform(X_test)

#### Score IDF de algunas palabras

In [13]:
words_example = [
    'skin',
    'product',
    'shampoo',
    'lotion',
    'dry',
]

In [14]:
vocab_idf = dict(zip(feat_extr.get_feature_names_out(), feat_extr.idf_))

print('{0:20}{1:20}'.format('Palabra', 'IDF'))
for word in words_example:
    if word not in vocab_idf:
        print('{0:20}{1:20}'.format(word, 'OOV'))
    else:
        print('{0:20}{1:2.3f}'.format(word, vocab_idf[word]))

Palabra             IDF                 
skin                2.983
product             2.242
shampoo             4.410
lotion              4.858
dry                 3.702


#### Voy a probar un modelo Regresión Logística en primer lugar:

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [16]:
# Barremos varios valores de c
c_params = [0.01, 0.05, 0.25, 0.5, 1, 10, 100]

lr_train_acc = list()
lr_test_acc = list()
for c in c_params:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=1000)
    lr.fit(X_train_, y_train)

    lr_train_predict = lr.predict(X_train_)
    lr_test_predict = lr.predict(X_test_)

    print ("Test Accuracy for C={}: {}".format(c, accuracy_score(y_test, lr_test_predict)))

    lr_train_acc.append(accuracy_score(y_train, lr_train_predict))
    lr_test_acc.append(accuracy_score(y_test, lr_test_predict))

print("Accuracy en train con el valor de 'c' seleccionado: {} ".format(max(lr_train_acc)))
print("Accuracy en test con el valor de 'c' seleccionado: {} ".format(max(lr_test_acc)))

Test Accuracy for C=0.01: 0.8111867120272164
Test Accuracy for C=0.05: 0.8312987792675606
Test Accuracy for C=0.25: 0.8556133680208124
Test Accuracy for C=0.5: 0.8601160696417851
Test Accuracy for C=1: 0.864018411046628
Test Accuracy for C=10: 0.8585151090654393
Test Accuracy for C=100: 0.8506103662197319
Accuracy en train con el valor de 'c' seleccionado: 0.9038756587285705 
Accuracy en test con el valor de 'c' seleccionado: 0.864018411046628 


#### Ahora un Árbol Clasificador:

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

In [18]:
n_estimators = 300
l_rate = 0.1

gb = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=l_rate, max_depth=3, random_state=42)

gb.fit(X_train_, y_train)

print("GradientBoosting Classifier Train Accuracy: {}".format(gb.score(X_train_, y_train)))
print("GradientBoosting Classifier Test Accuracy: {}".format(gb.score(X_test_, y_test)))

gb_train_predict = gb.predict(X_train_)
gb_test_predict = gb.predict(X_test_)


GradientBoosting Classifier Train Accuracy: 0.8481422186645321
GradientBoosting Classifier Test Accuracy: 0.8219931959175505


### Algunas métricas relevantes para comparar ambos modelos y elegir  uno

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,  precision_recall_curve # Reporte

In [20]:
print('Logistic Regression Confussion matrix:\n{}'.format(confusion_matrix(y_test, lr_test_predict)))
print('Logistic Regression Classification report:\n{}'.format(classification_report(y_test, lr_test_predict)))
print('Logistic Regression Accuracy score:{}'.format(accuracy_score(y_test, lr_test_predict)))

Logistic Regression Confussion matrix:
[[4255  717]
 [ 776 4246]]
Logistic Regression Classification report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85      4972
           1       0.86      0.85      0.85      5022

    accuracy                           0.85      9994
   macro avg       0.85      0.85      0.85      9994
weighted avg       0.85      0.85      0.85      9994

Logistic Regression Accuracy score:0.8506103662197319


In [21]:
print('GradientBoosting Classifier Confussion matrix:\n{}'.format(confusion_matrix(y_test, gb_test_predict)))
print('GradientBoosting Classifier Classification report:\n{}'.format(classification_report(y_test, gb_test_predict)))
print('GradientBoosting Classifier Accuracy score:{}'.format(accuracy_score(y_test, gb_test_predict)))

GradientBoosting Classifier Confussion matrix:
[[4291  681]
 [1098 3924]]
GradientBoosting Classifier Classification report:
              precision    recall  f1-score   support

           0       0.80      0.86      0.83      4972
           1       0.85      0.78      0.82      5022

    accuracy                           0.82      9994
   macro avg       0.82      0.82      0.82      9994
weighted avg       0.82      0.82      0.82      9994

GradientBoosting Classifier Accuracy score:0.8219931959175505


Evaluando métricas como Accuracy y el f1-score, que es muy importante en problemas de clasificación binaria, escojo el modelo de **Regresión Logística**, ya que tiene mejor resultado en estos parámetros. Aparte, resulta el modelo más sencillo para un problema como el planteado

Es importante señalar además que puedo fijarme en la métrica del Accuracy porque he "forzado" a que el problema esté balanceado a nivel de clases: tenemos la misma cantidad de valores para ambas etiquetas.

Para calcular métricas avanzadas que se piden en el siguiente apartado, calculo la probabilidad estimada en el conjunto de test con mi modelo de Regresión Logística:

In [22]:
y_prob = lr.predict_proba(X_test_)[:,1]

Voy a probar también con un modelo XGBoost Clasificador:

In [23]:
!pip install xgboost

In [24]:
from xgboost import XGBClassifier

xgbt = XGBClassifier(random_state=0, max_depth=3,learning_rate=0.01, n_estimators=500)
xgbt.fit(X_train_, y_train)

print("Train: ", xgbt.score(X_train_, y_train))
print("Test: ", xgbt.score(X_test_, y_test))

Train:  0.7646254419318258
Test:  0.755953572143286


In [25]:
xgbt_train_predict = xgbt.predict(X_train_)
xgbt_test_predict = xgbt.predict(X_test_)

In [26]:
print('XGBoost Classifier Confussion matrix:\n{}'.format(confusion_matrix(y_test, xgbt_test_predict)))
print('XGBoost Classifier Classification report:\n{}'.format(classification_report(y_test, xgbt_test_predict)))
print('XGBoost Classifier Accuracy score:{}'.format(accuracy_score(y_test, xgbt_test_predict)))

XGBoost Classifier Confussion matrix:
[[4126  846]
 [1593 3429]]
XGBoost Classifier Classification report:
              precision    recall  f1-score   support

           0       0.72      0.83      0.77      4972
           1       0.80      0.68      0.74      5022

    accuracy                           0.76      9994
   macro avg       0.76      0.76      0.75      9994
weighted avg       0.76      0.76      0.75      9994

XGBoost Classifier Accuracy score:0.755953572143286


### Para mi conjunto de datos, confirmo que el modelo que mejor resultados me da en todas las métricas es el de Regresión Logística.

### Pruebo un modelo en Deep Learning

Importo librerías:

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SimpleRNN

Lo primero es *tokenizar* las reseñas de texto usando el `Tokenizer` de Keras, que lo que hace es transformar cada texto en una secuencia de identificadores (números enteros), hasta un máximo de palabras definido en el parámetro `num_words`

In [28]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
# Se lo aplicamos al conjunto de reviews
tokenizer.fit_on_texts(proc_df['processedReview'])

Para definir el "padding" que queremos aplicar, es decir, el número de palabras que va a tener cada review, tenemos que pasar estas secuencias a textos usando `texts_to_sequences`. Este método de `Tokenizer` sólo va a tener en cuenta la cantidad y palabras que le hemos definido en el paso anterior

In [29]:
sequences = tokenizer.texts_to_sequences(proc_df['processedReview'])

In [30]:
max_seq_length = max(len(seq) for seq in sequences)

In [31]:
# Ahora aplicamos pad_sequences
padded_sequences = pad_sequences(sequences, max_seq_length)

Ya podemos dividir en los conjuntos de Train y Test

In [32]:
# Creamos arrays
X = np.array(padded_sequences)
y = np.array(proc_df['sentiment_label'])

Dividimos el dataset en Train y Test:

In [33]:
X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(
    X, # Reviews procesadas
    y, # Cogemos la variable binaria
    train_size=0.75,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

Modelo una red neuronal muy sencillita, con una capa de Embedding y una RNN, como para probar qué tal funciona

In [34]:
embedding_size = 32
model_rnn = Sequential()
model_rnn.add(Embedding(5000, embedding_size, input_length=max_seq_length))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(1, activation='sigmoid'))

model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


Entrenamos la red neuronal con los datos de Train. **NOTA**: Me tardaba mucho tiempo el entrenamiento, así que sólo he dejado 2 épocas. Lo correcto sería aplicar más o menos `epochs = 5`

In [35]:
model_rnn.fit(X_train_dl, y_train_dl, epochs=2, batch_size=32, validation_split=0.2)

Epoch 1/2
750/750 [==============================] - 1134s 1s/step - loss: 0.6554 - accuracy: 0.5977 - val_loss: 0.6183 - val_accuracy: 0.6437
Epoch 2/2
750/750 [==============================] - 1138s 2s/step - loss: 0.5651 - accuracy: 0.6950 - val_loss: 0.5393 - val_accuracy: 0.7150


Sacamos las métricas de este modelo previo, en *draft*:

In [36]:
loss, accuracy = model_rnn.evaluate(X_test_dl, y_test_dl)
print("Loss:", loss)
print("Accuracy:", accuracy)

313/313 [==============================] - 37s 119ms/step - loss: 0.5353 - accuracy: 0.7233
Loss: 0.5353431105613708
Accuracy: 0.723334014415741


Da unos resultados bastante buenos, en sólo dos épocas.



---



Guardo los datos del modelo elegido (Regresión Logística) para el siguiente apartado:

In [37]:
preds = pd.DataFrame({
    'y_test': y_test,
    'lr_test_pred': lr_test_predict,
    'y_test_prob' : y_prob
})

preds.to_csv('/content/drive/My Drive/Práctica NLP Najli YE/preds.csv', index=False)



---

